### Residual block

In [16]:
import numpy as np
import matplotlib.pyplot as plt

x = np.random.randn(5,1)
w = np.random.randn(5,5) * 0.01
b = np.random.randn(5,1)

def linear(x):
    return w @ x + b

def relu(x):
    x[np.where(x < 0)] = 0
    return x

x = linear(x)
x = relu(x)

a = x           # ──┐
                #   │
x = linear(x)   #   │
x = relu(x)     #   a
                #   │
x = linear(x)   #   │
x = relu(x + a) # ──┘

a = x           # ──┐
                #   │
x = linear(x)   #   │
x = relu(x)     #   a
                #   │
x = linear(x)   #   │
x = relu(x + a) # ──┘

a = x           # ──┐
                #   │
x = linear(x)   #   │
x = relu(x)     #   a
                #   │
x = linear(x)   #   │
x = relu(x + a) # ──┘

x

array([[0.3579266 ],
       [0.94916421],
       [0.        ],
       [0.99054694],
       [0.        ]])

### Inception block

<img src="https://production-media.paperswithcode.com/methods/Screen_Shot_2020-06-22_at_3.22.39_PM.png" width="1000px" />

In [12]:
import torch
import torch.nn as nn

x = torch.randn(1, 192, 28, 28) # (m,c,h,w)
x.shape

torch.Size([1, 192, 28, 28])

In [13]:
# (b) Inception module with dimension reductions

def conv_1(x, in_dim, out_dim):
    model = nn.Sequential(
        nn.Conv2d(in_dim, out_dim, 1, 1),
        nn.ReLU(),
    )
    return model(x)

def conv_1_3(x, in_dim, mid_dim, out_dim):
    model = nn.Sequential(
        nn.Conv2d(in_dim, mid_dim, 1, 1),
        nn.ReLU(),
        nn.Conv2d(mid_dim, out_dim, 3, 1, 1),
        nn.ReLU(),
    )
    return model(x)

def conv_1_5(x, in_dim, mid_dim, out_dim):
    model = nn.Sequential(
        nn.Conv2d(in_dim, mid_dim, 1, 1),
        nn.ReLU(),
        nn.Conv2d(mid_dim, out_dim, 5, 1, 2),
        nn.ReLU(),
    )
    return model(x)

def max_3_1(x, in_dim, out_dim):
    model = nn.Sequential(
        nn.MaxPool2d(3, 1, 1),
        nn.Conv2d(in_dim, out_dim, 1, 1),
        nn.ReLU(),
    )
    return model(x)

out_1 = conv_1(x, 192, 96)
out_2 = conv_1_3(x, 192, 64, 96)
out_3 = conv_1_5(x, 192, 16, 32)
out_4 = max_3_1(x, 192, 32)
x = torch.cat([out_1, out_2, out_3, out_4], 1)
x.shape

torch.Size([1, 256, 28, 28])

### MobileNet
Normal convolutions vs. depthwise-separable convolutions

<img src="https://production-media.paperswithcode.com/methods/Screen_Shot_2020-05-31_at_10.30.20_PM.png" width="600px" />

In [27]:
import torch
import torch.nn as nn

x = torch.randn(1, 3, 28, 28)
conv = nn.Conv2d(in_channels=3,
                 out_channels=1,
                 kernel_size=3,
                 padding=1,
                 groups=1)
x = conv(x)
print(x.shape)

x = torch.randn(1, 3, 28, 28)
conv_depthwise = nn.Conv2d(in_channels=3,
                           out_channels=3,
                           kernel_size=3,
                           padding=1,
                           groups=3)
conv_pointwise = nn.Conv2d(in_channels=3,
                           out_channels=1,
                           kernel_size=1)
x = conv_depthwise(x)
x = conv_pointwise(x)
print(x.shape)

torch.Size([1, 1, 28, 28])
torch.Size([1, 1, 28, 28])


MobileNet v2 = MobileNet v1 + ResNet

-> [Expansion -> Depthwise -> Pointwise] -> [Expansion -> Depthwise -> Pointwise] ->

In [34]:
x = torch.randn(1,3,28,28)

conv_expansion = nn.Conv2d(
    in_channels=3,
    out_channels=18,
    kernel_size=1,
    groups=1)

conv_depthwise = nn.Conv2d(
    in_channels=18,
    out_channels=18,
    kernel_size=3,
    padding=1,
    groups=18)

conv_pointwise = nn.Conv2d(
    in_channels=18,
    out_channels=3,
    kernel_size=1,
    groups=1)

relu = nn.ReLU()

def bottleneck(x):
    _x = x
    x = conv_expansion(x)
    x = relu(x)
    x = conv_depthwise(x)
    x = relu(x)
    x = conv_pointwise(x)
    x = relu(x + _x)
    return x

def mobilenetv2(x):
    for _ in range(17):
        x = bottleneck(x)
    return x

mobilenetv2(x).shape

torch.Size([1, 3, 28, 28])

### IOU (Intersection Over Union)
Basically a f1 score

<img src="https://machinelearningspace.com/wp-content/uploads/2023/01/IOU2.jpg" width="600px" />